In [2]:

from __future__ import print_function

import sys

# sys.path.extend(['/home/jogi/git/repository/dcnn_acdc'])
sys.path.extend(['/home/jorg/repository/dcnn_acdc'])

import matplotlib
import matplotlib.pyplot as plt

from matplotlib import cm


from datetime import datetime
from pytz import timezone
import itertools
import copy

import SimpleITK as sitk
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init

import numpy as np
import dill
import os
import glob
from collections import OrderedDict

from in_out.load_data import ACDC2017DataSet
from config.config import config
from utils.experiment import Experiment, ExperimentHandler
from utils.batch_handlers import TwoDimBatchHandler
from utils.test_handler import ACDC2017TestHandler
from plotting.learning_curves import loss_plot
from plotting.uncertainty_plots import analyze_slices
from utils.generate_uncertainty_maps import UncertaintyMapsGenerator

from models.dilated_cnn import BaseDilated2DCNN

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Constants aka fixed variables
ROOT_DIR = "/home/jorg/repository/dcnn_acdc"
LOG_DIR = os.path.join(ROOT_DIR, "logs")
CYCLE_OFFSET = 100000
# set seeds
SEED = 4325
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.enabled = True
np.random.seed(SEED)

# IMPORTANT - THE FOLD WE'RE USING FOR TESTING. NOTE: Training and testing fold should be the same
# we raise an error in the test_ensemble method if this is not the case!
TEST_FOLD = 0

# baseline: 
exp_base = {2: "20180328_10_53_16_dcnnv1_150000E_lr2e02",
            0: "20180330_09_56_39_dcnnv1_150000E_lr2e02"}
exp_base_model = os.path.join(LOG_DIR, exp_base[TEST_FOLD])
# Jelmers base model with brier-score
# Fold 2: 20180329_08_02_33_dcnnv1_150000E_lr2e04
# Fold 0: 20180331_09_48_22_dcnnv1_150000E_lr2e02
exp_base_bs = {2: "20180329_08_02_33_dcnnv1_150000E_lr2e04",
               0: "20180331_09_48_22_dcnnv1_150000E_lr2e02"}
exp_base_bs_model = os.path.join(LOG_DIR, exp_base_bs[TEST_FOLD])
# MC dropout models
# Fold 2: 20180328_10_54_36_dcnn_mcv1_150000E_lr2e02
# Fold 0: 20180330_09_56_01_dcnn_mcv1_150000E_lr2e02
exp_mc005 = {2: "20180328_10_54_36_dcnn_mcv1_150000E_lr2e02",
             0: "20180330_09_56_01_dcnn_mcv1_150000E_lr2e02"}
exp_mc005_model = os.path.join(LOG_DIR, exp_mc005[TEST_FOLD])
# Fold 2: 20180329_08_04_12_dcnn_mcv1_150000E_lr2e04
# Fold 0: 20180331_09_49_14_dcnn_mcv1_150000E_lr2e02
exp_mc005_bs = {2: "20180329_08_04_12_dcnn_mcv1_150000E_lr2e04",
                0: "20180331_09_49_14_dcnn_mcv1_150000E_lr2e02"}
exp_mc005_bs_model = os.path.join(LOG_DIR, exp_mc005_bs[TEST_FOLD])

# Fold 2: 20180328_10_54_55_dcnn_mcv1_150000E_lr2e02
# Fold 0: 20180330_09_56_15_dcnn_mcv1_150000E_lr2e02
exp_mc01 = {2: "20180328_10_54_55_dcnn_mcv1_150000E_lr2e02",
            0: "20180330_09_56_15_dcnn_mcv1_150000E_lr2e02"}
exp_mc01_model = os.path.join(LOG_DIR, exp_mc01[TEST_FOLD])
# Fold 2: 20180329_08_03_29_dcnn_mcv1_150000E_lr2e04
# Fold 0: 20180331_09_48_48_dcnn_mcv1_150000E_lr2e02
exp_mc01_bs = {2: "20180329_08_03_29_dcnn_mcv1_150000E_lr2e04",
               0: "20180331_09_48_48_dcnn_mcv1_150000E_lr2e02"}
exp_mc01_bs_model = os.path.join(LOG_DIR, exp_mc01_bs[TEST_FOLD])

# Fold 2: 20180329_08_26_03_dcnn_mcv1_150000E_lr2e02
# Fold 0: 20180404_09_06_37_dcnn_mcv1_150000E_lr2e02
exp_mc02 = {2: "20180329_08_26_03_dcnn_mcv1_150000E_lr2e02",
            0: "20180404_09_06_37_dcnn_mcv1_150000E_lr2e02"}
exp_mc02_model = os.path.join(LOG_DIR, exp_mc02[TEST_FOLD])
# Fold 2: 20180331_10_03_03_dcnn_mcv1_150000E_lr2e04
# Fold 0: 20180404_09_12_48_dcnn_mcv1_150000E_lr2e04
exp_mc02_bs = {2: "20180331_10_03_03_dcnn_mcv1_150000E_lr2e04",
               0: "20180404_09_12_48_dcnn_mcv1_150000E_lr2e04"}
exp_mc02_bs_model = os.path.join(LOG_DIR, exp_mc02_bs[TEST_FOLD])

# Fold 2: 20180329_08_42_47_dcnn_mcv1_150000E_lr2e02
# Fold 0: 20180404_09_05_00_dcnn_mcv1_150000E_lr2e02
exp_mc03 = {2: "20180329_08_42_47_dcnn_mcv1_150000E_lr2e02",
            0: "20180404_09_05_00_dcnn_mcv1_150000E_lr2e02"}
exp_mc03_model = os.path.join(LOG_DIR, exp_mc03[TEST_FOLD])
# Fold 2: 20180331_10_02_23_dcnn_mcv1_150000E_lr2e04
# Fold 0: 20180405_08_32_07_dcnn_mcv1_150000E_lr2e04
exp_mc03_bs = {2: "20180331_10_02_23_dcnn_mcv1_150000E_lr2e04",
               0: "20180405_08_32_07_dcnn_mcv1_150000E_lr2e04"}
exp_mc03_bs_model = os.path.join(LOG_DIR, exp_mc03_bs[TEST_FOLD])

In [ ]:
exper_mc005 = ExperimentHandler.load_experiment(exp_mc005_model)
exper_hdl_mc005 = ExperimentHandler(exper_mc005, use_logfile=False)
exper_hdl_mc005.set_root_dir(ROOT_DIR)
exper_hdl_mc005.set_model_name("MC-dropout fold-{} (p={})".format(exper_hdl_mc005.exper.run_args.fold_ids[0],
    exper_hdl_mc005.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc005.exper.model_name))

exper_mc005_bs = ExperimentHandler.load_experiment(exp_mc005_bs_model)
exper_hdl_mc005_bs = ExperimentHandler(exper_mc005_bs, use_logfile=False)
exper_hdl_mc005_bs.set_root_dir(ROOT_DIR)
exper_hdl_mc005_bs.set_model_name("MC-BS-dropout fold-{} (p={})".format(
    exper_hdl_mc005_bs.exper.run_args.fold_ids[0],
    exper_hdl_mc005_bs.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc005_bs.exper.model_name))

exper_mc01 = ExperimentHandler.load_experiment(exp_mc01_model)
exper_hdl_mc01 = ExperimentHandler(exper_mc01, use_logfile=False)
exper_hdl_mc01.set_root_dir(ROOT_DIR)
exper_hdl_mc01.set_model_name("MC-dropout fold-{} (p={})".format(exper_hdl_mc01.exper.run_args.fold_ids[0],
    exper_hdl_mc01.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc01.exper.model_name))

exper_mc01_bs = ExperimentHandler.load_experiment(exp_mc01_bs_model)
exper_hdl_mc01_bs = ExperimentHandler(exper_mc01_bs, use_logfile=False)
exper_hdl_mc01_bs.set_root_dir(ROOT_DIR)
exper_hdl_mc01_bs.set_model_name("MC-BS-dropout fold-{} (p={})".format(
    exper_hdl_mc01_bs.exper.run_args.fold_ids[0],
    exper_hdl_mc01_bs.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc01_bs.exper.model_name))


exper_mc02 = ExperimentHandler.load_experiment(exp_mc02_model)
exper_hdl_mc02 = ExperimentHandler(exper_mc02, use_logfile=False)
exper_hdl_mc02.set_root_dir(ROOT_DIR)
exper_hdl_mc02.set_model_name("MC-dropout fold-{} (p={})".format(exper_hdl_mc02.exper.run_args.fold_ids[0],
    exper_hdl_mc02.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc02.exper.model_name))

exper_mc02_bs = ExperimentHandler.load_experiment(exp_mc02_bs_model)
exper_hdl_mc02_bs = ExperimentHandler(exper_mc02_bs, use_logfile=False)
exper_hdl_mc02_bs.set_root_dir(ROOT_DIR)
exper_hdl_mc02_bs.set_model_name("MC-BS-dropout fold-{} (p={})".format(
    exper_hdl_mc02_bs.exper.run_args.fold_ids[0],
    exper_hdl_mc02_bs.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc02_bs.exper.model_name))


exper_mc03 = ExperimentHandler.load_experiment(exp_mc03_model)
exper_hdl_mc03 = ExperimentHandler(exper_mc03, use_logfile=False)
exper_hdl_mc03.set_root_dir(ROOT_DIR)
exper_hdl_mc03.set_model_name("MC-dropout fold-{} (p={})".format(exper_hdl_mc03.exper.run_args.fold_ids[0],
    exper_hdl_mc03.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc03.exper.model_name))


exper_mc03_bs = ExperimentHandler.load_experiment(exp_mc03_bs_model)
exper_hdl_mc03_bs = ExperimentHandler(exper_mc03_bs, use_logfile=False)
exper_hdl_mc03_bs.set_root_dir(ROOT_DIR)
exper_hdl_mc03_bs.set_model_name("MC-BS-dropout fold-{} (p={})".format(
        exper_hdl_mc03_bs.exper.run_args.fold_ids[0],
        exper_hdl_mc03_bs.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc03_bs.exper.model_name))


try:
    del test_set
except:
    pass

test_set = ACDC2017TestHandler(exper_config=config, 
                               search_mask=config.dflt_image_name + ".mhd", fold_ids=[TEST_FOLD],
                               debug=False, batch_size=5, use_cuda=True, val_only=True)

In [ ]:
from utils.generate_uncertainty_maps import UncertaintyMapsGenerator
mc_samples = 10
exper_handler = exper_hdl_mc005
exper_handler.logger = None

maps_generator = UncertaintyMapsGenerator(exper_handler, test_set=test_set, verbose=False, mc_samples=mc_samples)
maps_generator()

In [ ]:
from sklearn import mixture
from matplotlib.colors import LogNorm

In [ ]:
def get_uncertainty_maps(testset, image_num, c_phase=0, slice_id=0, uncertainty_type="bald"):
    if uncertainty_type == "bald":
        uncertainty_map = testset.bald_maps[image_num][c_phase, :, :, slice_id]
    else:
        uncertainty_map = testset.uncertainty_maps[image_num][c_phase, :, :, slice_id]
    
    return uncertainty_map        

heat_map = get_uncertainty_maps(exper_handler.test_results, image_num=0, c_phase=1, slice_id=0)
print(heat_map.shape)

un_data = heat_map.ravel()

del clf
gmm = mixture.GaussianMixture(n_components=10, covariance_type='full')
gmm.fit(heat_map)


# plt.imshow(z+10, extent=(np.amin(x), np.amax(x), np.amin(y), np.amax(y)),
#        cmap=cm.hot, norm=LogNorm())

# plt.colorbar()
# plt.show()

In [ ]:
# display predicted scores by the model as a contour plot
x = np.linspace(0, heat_map.shape[0])
y = np.linspace(0, heat_map.shape[1])
X, Y = np.meshgrid(x, y)
XX = np.array([X.ravel(), Y.ravel()]).T
Z = -clf.score_samples(XX)
Z = Z.reshape(X.shape)
CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0),
                 levels=np.logspace(0, 3, 10))
CB = plt.colorbar(CS, shrink=0.8, extend='both')

plt.title('Negative log-likelihood predicted by a GMM')
plt.axis('tight')
plt.show()_mc005

In [1]:
clf = mixture.GaussianMixture(n_components=2, covariance_type='full')
clf.fit(X_train)

# display predicted scores by the model as a contour plot
x = np.linspace(-20., 30.)
y = np.linspace(-20., 40.)
X, Y = np.meshgrid(x, y)
XX = np.array([X.ravel(), Y.ravel()]).T
Z = -clf.score_samples(XX)
Z = Z.reshape(X.shape)

CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0),
                 levels=np.logspace(0, 3, 10))
CB = plt.colorbar(CS, shrink=0.8, extend='both')
plt.scatter(X_train[:, 0], X_train[:, 1], .8)

plt.title('Negative log-likelihood predicted by a GMM')
plt.axis('tight')
plt.show()

NameError: name 'mixture' is not defined

In [22]:
test_set = ACDC2017TestHandler(exper_config=config, 
                               search_mask=config.dflt_image_name + ".mhd", fold_ids=[0],
                               debug=False, batch_size=5, use_cuda=True, val_only=True)

  0%|          | 0/5 [00:00<?, ?it/s]

INFO - File list contains 50 files, hence 25 patients
0 - /home/jorg/repository/dcnn_acdc/data/Folds/fold0/validate/images/patient016_frame01.mhd
1 - /home/jorg/repository/dcnn_acdc/data/Folds/fold0/validate/images/patient016_frame12.mhd


 20%|██        | 1/5 [00:00<00:02,  1.73it/s]/home/jorg/anaconda2/envs/pyt_0_3_0/lib/python2.7/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


2 - /home/jorg/repository/dcnn_acdc/data/Folds/fold0/validate/images/patient017_frame01.mhd
3 - /home/jorg/repository/dcnn_acdc/data/Folds/fold0/validate/images/patient017_frame09.mhd


 40%|████      | 2/5 [00:01<00:01,  1.71it/s]

...

6 - /home/jorg/repository/dcnn_acdc/data/Folds/fold0/validate/images/patient019_frame01.mhd
7 - /home/jorg/repository/dcnn_acdc/data/Folds/fold0/validate/images/patient019_frame11.mhd


 80%|████████  | 4/5 [00:02<00:00,  1.84it/s]

8 - /home/jorg/repository/dcnn_acdc/data/Folds/fold0/validate/images/patient020_frame01.mhd
9 - /home/jorg/repository/dcnn_acdc/data/Folds/fold0/validate/images/patient020_frame11.mhd


100%|██████████| 5/5 [00:02<00:00,  1.78it/s]

INFO - Successfully loaded 5 ED/ES patient pairs


In [21]:
ROOT_DIR = "/home/jorg/repository/dcnn_acdc"
LOG_DIR = os.path.join(ROOT_DIR, "logs")
exp_model = os.path.join(LOG_DIR, "20180330_09_56_01_dcnn_mcv1_150000E_lr2e02")
exper = ExperimentHandler.load_experiment(exp_model)
exper_hdl = ExperimentHandler(exper, use_logfile=False)
exper_hdl.set_root_dir(ROOT_DIR)
exper_hdl.set_model_name("MC-dropout fold-{} (p={})".format(exper_hdl.exper.run_args.fold_ids[0],
    exper_hdl.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl.exper.model_name))

Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_01_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.dll
Model name MC-dropout fold-0 (p=0.05)


In [76]:
u_threshold = 0.1
mc_samples = 10
exper_handler = exper_hdl
exper_handler.logger = None

maps_generator = UncertaintyMapsGenerator(exper_handler, test_set=test_set, verbose=False, mc_samples=mc_samples,
                                          u_threshold=u_threshold)
maps_generator()

  0%|          | 0/5 [00:00<?, ?it/s]

INFO - loading model MC-dropout fold-0 (p=0.05)
INFO - Starting to generate uncertainty maps for 5 images using 10 samples and u-threshold


 20%|██        | 1/5 [00:09<00:39,  9.86s/it]

INFO - Successfully saved maps to /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_01_dcnn_mcv1_150000E_lr2e02/u_maps/patient016_umaps.npz


 40%|████      | 2/5 [00:20<00:31, 10.47s/it]

...

INFO - Successfully saved maps to /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_01_dcnn_mcv1_150000E_lr2e02/u_maps/patient018_umaps.npz


 80%|████████  | 4/5 [00:40<00:10, 10.05s/it]

INFO - Successfully saved maps to /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_01_dcnn_mcv1_150000E_lr2e02/u_maps/patient019_umaps.npz


100%|██████████| 5/5 [00:51<00:00, 10.33s/it]

INFO - Successfully saved maps to /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_01_dcnn_mcv1_150000E_lr2e02/u_maps/patient020_umaps.npz
INFO - Total duration of generation process 51.65 secs


In [77]:
from utils.generate_uncertainty_maps import UncertaintyMapsGenerator

batch_uncty = UncertaintyMapsGenerator.load_uncertainty_maps(exper_handler=exper_handler)

/home/jorg/repository/dcnn_acdc/logs/20180330_09_56_01_dcnn_mcv1_150000E_lr2e02/u_maps/*_umaps.npz


In [79]:

batch_uncty.gen_normalized_stats()
batch_uncty.detect_outliers()
print(batch_uncty.outliers.keys())

[('patient020', 0), ('patient020', 7), ('patient020', 1), ('patient020', 2), ('patient020', 4), ('patient020', 6), ('patient016', 8), ('patient016', 9), ('patient017', 8), ('patient018', 7), ('patient019', 10)]


In [ ]:
exper_handler = exper_handler

image_range = None
u_type = "stddev"

analyze_slices(exper_handler, image_range=image_range, do_save=True, do_show=True, u_type=u_type,
               translate_img_range=True)